This notebook is ran in a docker container where the project directory (i.e. same directory as README.md) is located in `/code`, which is set below. If you run locally you'll need to set the path of your project directory accordingly.

In [1]:
%cd /code

/code


---

The `load_dotenv` function below loads all the variables found in the `.env` file as environment variables. You must have a `.env` file located in the project directory containing your OpenAI API key, in the following format.

```
OPENAI_API_KEY=sk-...
```

In [2]:
from dotenv import load_dotenv
from IPython.display import display, Markdown

load_dotenv()

def mprint(val: str):
    display(Markdown(val))

---

# Chains

**A `Chain` consists of individual links. Each link can be thought of as a task in a workflow (e.g. document search, web search, or chat model). Each link is a callable (either a function or a callable object) where the output of one link is the input of the next link. So a `Chain` is a simple mechanism that takes an input and sends the input to the first link, and the propegates the output of the first link to the second link, and so on, until the end of the chain is reached, and returns the final result.** 

A `Chain` object aggregates the history (messages, usage, etc) of all of the links. More specifically, it aggregiates all of the lists of `Record` objects for any link that has a `history` property, such as the OpenAIChat class. The `Chain` class provides various history properties (e.g. `history`, `usage_history`, `message_history`) depending on the type of `Record` returned by each link's history property (e.g. the `usage_history` property returns all of the `UsageRecord` objects across any link whose `history` property returns `UsageRecord`s).

---

## Simple Example

Here's a simple example demonstrating llm-chain. A more in-depth example with additional explanation can be found below.

- Ask the chat model a question ("What is the meaning of life?")
- The model responds, and the response is sent to the next link, which creates and returns a new prompt indicating that the link's input (which is the output from the last link; i.e. the model's response) should be summarized.
- The new prompt is sent to the next link, which is the chat model, and the response is returned.

In [28]:
from llm_chain.base import Chain
from llm_chain.models import OpenAIChat

chat_model = OpenAIChat(model_name='gpt-3.5-turbo')
# each link is a callable where the output of one link is the input to the next link
chain = Chain(links=[
    chat_model,
    lambda x: f"Summarize the following in two sentences: ```{x}```",
    chat_model,
])
chain("What is the meaning of life?")

"The meaning of life is a philosophical question that has been debated for centuries with no definitive answer. It varies depending on one's beliefs, values, and experiences and is ultimately a personal and subjective concept."

In [32]:
print(f"Cost:   ${chain.cost:.4f}")
print(f"Tokens: {chain.total_tokens:,}")

Cost:   $0.0007
Tokens: 395


In [30]:
chain.message_history

[MessageRecord(uuid='4eb3317a-37c7-4029-b394-07698aa8454f', timestamp='2023-06-26 22:04:05.331', metadata={'model_name': 'gpt-3.5-turbo'}, total_tokens=120, cost=0.000227, prompt='What is the meaning of life?', response="The meaning of life is a philosophical question that has been debated by scholars, theologians, and philosophers for centuries. There is no one definitive answer to this question, as it can vary depending on one's beliefs, values, and experiences. Some people believe that the meaning of life is to seek happiness, while others believe it is to fulfill a specific purpose or destiny. Ultimately, the meaning of life is a personal and subjective concept that each individual must determine for themselves.", prompt_tokens=26, response_tokens=94),
 MessageRecord(uuid='28787fff-0e62-4bf2-a1c4-a4046d6f659a', timestamp='2023-06-26 22:04:07.090', metadata={'model_name': 'gpt-3.5-turbo'}, total_tokens=275, cost=0.0004325, prompt="Summarize the following in two sentences: ```The mea

In [31]:
mprint(chain.message_history[0].prompt)
mprint(chain.message_history[0].response)
mprint(chain.message_history[1].prompt)
mprint(chain.message_history[1].response)

What is the meaning of life?

The meaning of life is a philosophical question that has been debated by scholars, theologians, and philosophers for centuries. There is no one definitive answer to this question, as it can vary depending on one's beliefs, values, and experiences. Some people believe that the meaning of life is to seek happiness, while others believe it is to fulfill a specific purpose or destiny. Ultimately, the meaning of life is a personal and subjective concept that each individual must determine for themselves.

Summarize the following in two sentences: ```The meaning of life is a philosophical question that has been debated by scholars, theologians, and philosophers for centuries. There is no one definitive answer to this question, as it can vary depending on one's beliefs, values, and experiences. Some people believe that the meaning of life is to seek happiness, while others believe it is to fulfill a specific purpose or destiny. Ultimately, the meaning of life is a personal and subjective concept that each individual must determine for themselves.```

The meaning of life is a philosophical question that has been debated for centuries with no definitive answer. It varies depending on one's beliefs, values, and experiences and is ultimately a personal and subjective concept.

---

## In-depth Example

In the following example, we will ask a chat model a question, and provide the model with additional context based on the most relevant text we find in a web-search. For the sake of the example, we will ask the chat model to the answer it provides and summarize that answer.  The workflow is defined as follows:


|   input   |  output  |   link/task   |
|-----------|----------|----------|
|  `None` |   str    |  Ask a question: e.g. `"What is the meaning of life?"`  |
|    str    |   urls   |  Do a web-search via `DuckDuckGo` based on the question.  |
|   urls    | Documents|  Take the urls, scrape the corresponding web-pages, and convert to a list of `Document` objects  |
| Documents | Documents|  Split the `Document` objects into smaller chunks (also `Document` objects)  |
| Documents | `None` |  Store the Documents in a `Chroma` document index so that we can lookup the most relevant documents |
| `None`  |   str    |  Return the original question `What is the meaning of life?` |
|    str    |   str    |  Take the original question, lookup the most relevant documents, and construct the corresponding prompt with the documents injected into the prompt. |
|    str    |   str    |  Pass the prompt to the chat model; receive a response |
|    str    |   str    |  Construct a new prompt, containing the response and a request to summarize the response. |
|    str    |   str    |  Pass the new prompt to the chat model; receive a response |

**Notice how the output of one link matches the input of the next link. This means you could swap out any of the objects we use below with your own custom function or class and the only requirement is that the input/output matches for that particular link.**



### Initial objects in chain

Let's define the objects and functions we are going to include in the chain. We're including many links/tasks so that we can see how flexible our chain can be.

See the [tools.ipynb](https://github.com/shane-kercheval/llm-chain/tree/main/examples/tools.ipynb) notebook for examples of how the various helper classes below (e.g. `DuckDuckGoSearch`, `scrape_url`, etc.) are used.

#### Quick note on the `Value` object.

The only bit of magic below is that we're using a `Value` object to cache the initial question, feed it into the web-search, and then inject it back into the chain at a later point (passing it to the prompt-template). A `Value` object is a callable that, when called with a value caches and returns the same value, and called without a value, simply returns the previously cached value. When you understand what it's doing, it's not magic at all. Here's a simple example:

In [3]:
from llm_chain.base import Value
cache = Value()
result = cache("This is a value")  # calling the object with a value caches and returns the value
print(result)  # the value in `result` is whatever was passed in to the Value object
print(cache())  # calling the object without a parameter returns the cached value
result = cache("new value")
print(result)
print(cache())

This is a value
This is a value
new value
new value


In [4]:
from llm_chain.base import Document, Chain, Value
from llm_chain.models import OpenAIEmbeddings, OpenAIChat
from llm_chain.tools import DuckDuckGoSearch, scrape_url, split_documents
from llm_chain.indexes import ChromaDocumentIndex
from llm_chain.prompt_templates import DocSearchTemplate

# Seach DuckDuckGo based on initial question (passed into the chain)
duckduckgo_search = DuckDuckGoSearch(top_n=3)

# define a function that takes the links from the web-search, scrapes the web-pages,
# and then creates Document objects from the text of each web-page
def scrape_urls(search_results):
    """
    For each url (i.e. `href` in `search_results`):
    - extracts text
    - replace new-lines with spaces
    - create a Document object
    """
    return [
        Document(content=scrape_url(x['href']).replace('\n', ' '))
        for x in search_results
    ]

# Embeddings model used for document index/search (the documents created from the web-search)
embeddings_model = OpenAIEmbeddings(model_name='text-embedding-ada-002')
document_index = ChromaDocumentIndex(embeddings_model=embeddings_model, n_results=3)
# DocSearchTemplate uses the ChromaDocumentIndex object to search for the most relevant documents
# (from the web-search) based on the intitial question (which it takes as an input)
prompt_template = DocSearchTemplate(doc_index=document_index, n_docs=3)
# Create a chat model without streaming.
non_streaming_chat = OpenAIChat(model_name='gpt-3.5-turbo')
# Create a chat model with streaming enabled via callback.
streaming_chat = OpenAIChat(
    model_name='gpt-3.5-turbo',
    streaming_callback=lambda x: print(x.response, end='|'),
)

# A `Value` object is a simple caching mechanism. It's a callable that, when passed a value, it
# caches and returns that value; and when called without a value, it returns the cached value.
# Below, it's being used to cache the original question, feed the question into the web-search
# (`DuckDuckGoSearch`), and then re-inject the question back in the chain and into the
# prompt-template (`DocSearchTemplate`).
question_1 = Value()
# This simple function takes the response from the original chat model and creates a prompt that
# asks the model to summarize the response.
question_2 = lambda x: f'Summarize the following in less than 20 words: "{x}"'

### Defining and Running the Chain

In [5]:
# each link is a callable where the output of one link is the input to the next
chain = Chain(links=[
    question_1,
    duckduckgo_search,
    scrape_urls,
    split_documents,  # split web-pages into smaller chunks; defaults to chunk-size of 500
    document_index,  # __call__ function calls add() if given a list of documents (which is returned by `split_documents`)
    question_1,
    prompt_template,
    non_streaming_chat,
    question_2,
    streaming_chat,
])
# the value passed into `chain()` is passed to the `initial_question` object (which
# is `Value` object and so it caches the value passed in and also returns it) which then gets
# passed to the `duckduckgo_search` object, and so on.
# the response of the final model is streamed, because our chat model has the streaming_callback
# set, but it should also match the response returned
response = chain("What is the meaning of life?")

The| meaning| of| life| is| subjective|,| with| popular| answers| including| realizing| potential| and| ideals|,| chasing| dreams|,| and| being| responsible|.|

In [6]:
response

'The meaning of life is subjective, with popular answers including realizing potential and ideals, chasing dreams, and being responsible.'

---

### Total costs and usage

The `Chain` object aggregates the costs/usage across all links that have a `history` property where that `history` property returns `UsageRecord` objects. In other words, any link that tracks its own history automatically gets counted towards the total usage within the Chain.

In [7]:
print(f"Cost:   ${chain.cost:.4f}")
print(f"Tokens: {chain.total_tokens:,}")

Cost:   $0.0055
Tokens: 45,491


### History

Similar to tracking the costs/usage, we can dig into the history at a more granular level.

As you can see below, the `history` property returns a list containing:

- a `SearchRecord` object capturing our original search query and results
- two `EmbeddingsRecord` records; the first corresponds to getting the embeddings of all of the chunks from the web-pages; the second corresponds to getting the embedding of our original question so that we can find the most relevant chunks; the `EmbeddingsRecord` is a `UsageRecord` so it contains costs/usage
- two `MessageRecord` records; the first corresponds to the first question (prompt & response) we made to the chat model; the second corresponds to our second query to the chat model asking it to summarize the first response; the `MessageRecord` is a `UsageRecord` so it contains costs/usage

In [8]:
chain.history

[SearchRecord(uuid='b9c0348c-9f98-4e5f-ada6-ddc49f96a710', timestamp='2023-06-26 15:41:55.549', metadata={}, query='What is the meaning of life?', results=[{'title': 'Meaning of life - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Meaning_of_life', 'body': 'The meaning of life, or the answer to the question: "What is the meaning of life?", pertains to the significance of living or existence in general. Many other related questions include: "Why are we here?", "What is life all about?", or "What is the purpose of existence?"'}, {'title': 'Life Definition & Meaning - Merriam-Webster', 'href': 'https://www.merriam-webster.com/dictionary/life', 'body': 'life: [noun] the quality that distinguishes a vital and functional being from a dead body. a principle or force that is considered to underlie the distinctive quality of animate beings. an organismic state characterized by capacity for metabolism (see metabolism 1), growth, reaction to stimuli, and reproduction.'}, {'title': 'What Is t

In [9]:
print(chain.history[3])
print(chain.history[4])

timestamp: 2023-06-26 15:42:15.408; prompt: "Answer the question ..."; response: "The meaning of life ...";  cost: $0.000793; total_tokens: 495; metadata: {'model_name': 'gpt-3.5-turbo'}
timestamp: 2023-06-26 15:42:16.911; prompt: "Summarize the follow..."; response: "The meaning of life ...";  cost: $0.000246; total_tokens: 156; metadata: {'model_name': 'gpt-3.5-turbo'}


#### `message_history`

We can use the `message_history` property which simply returns all of the history items that are of type `MessageRecord`

In [10]:
chain.message_history

[MessageRecord(uuid='ddf9c357-d2a5-490f-9ca5-2897ae89eb80', timestamp='2023-06-26 15:42:15.408', metadata={'model_name': 'gpt-3.5-turbo'}, total_tokens=495, cost=0.0007930000000000001, prompt='\nAnswer the question at the end of the text as truthfully and accurately as possible, based on the following information provided.\n\nHere is the information:\n\n```\nBy using free will, people must take an active role in the universal conflict, with good thoughts, good words and good deeds to ensure happiness and to keep chaos at bay.  Popular views "What is the meaning of life?" is a question many people ask themselves at some point during their lives, most in the context "What is the purpose of life?".[15] Some popular answers include:  To realize one\'s potential and ideals To chase dreams.[161] To live one\'s dreams.[162] To spend it for something that\n\nasked the most is “What is the meaning of life?” “Ultimately, man should not ask what the meaning of his life is, but rather must recogni

In [11]:
print(chain.message_history[0])  # same as chain.history[3]
print(chain.message_history[1])  # same as chain.history[4]

timestamp: 2023-06-26 15:42:15.408; prompt: "Answer the question ..."; response: "The meaning of life ...";  cost: $0.000793; total_tokens: 495; metadata: {'model_name': 'gpt-3.5-turbo'}
timestamp: 2023-06-26 15:42:16.911; prompt: "Summarize the follow..."; response: "The meaning of life ...";  cost: $0.000246; total_tokens: 156; metadata: {'model_name': 'gpt-3.5-turbo'}


#### First prompt to the chat model

You can see below that we used three chunks from our original web-search in the first prompt we sent to ChatGPT.

In [12]:
mprint(chain.message_history[0].prompt)


Answer the question at the end of the text as truthfully and accurately as possible, based on the following information provided.

Here is the information:

```
By using free will, people must take an active role in the universal conflict, with good thoughts, good words and good deeds to ensure happiness and to keep chaos at bay.  Popular views "What is the meaning of life?" is a question many people ask themselves at some point during their lives, most in the context "What is the purpose of life?".[15] Some popular answers include:  To realize one's potential and ideals To chase dreams.[161] To live one's dreams.[162] To spend it for something that

asked the most is “What is the meaning of life?” “Ultimately, man should not ask what the meaning of his life is, but rather must recognize that it is he who is asked. In a word, each man is questioned by life; and he can only answer to life by answering for his own life; to life he can only respond by being responsible.” Viktor Frankl We are all hungry for meaning, for purpose, for the feeling that our life is worth more than the sum of its parts. Luckily, humans are resourceful – we have

and anxiety) in an unconditional commitment to something finite and devotes that meaningful life to the commitment, despite the vulnerability inherent to doing so.[87] Arthur Schopenhauer answered: "What is the meaning of life?" by stating that one's life reflects one's will, and that the will (life) is an aimless, irrational, and painful drive. Salvation, deliverance, and escape from suffering are in aesthetic contemplation, sympathy for others, and asceticism.[88][89] For Friedrich Nietzsche,
```

Here is the question:

What is the meaning of life?


#### First response from the chat model

In [13]:
mprint("> " + chain.message_history[0].response)

> The meaning of life is a subjective question and there are various popular answers to it, such as realizing one's potential and ideals, chasing dreams, living one's dreams, or spending life for something. However, Viktor Frankl suggests that instead of asking what the meaning of life is, one should recognize that life is asking each individual and they can only answer by being responsible for their own life. Other philosophers like Arthur Schopenhauer and Friedrich Nietzsche have also provided their own perspectives on the meaning of life.

#### Second prompt to the chat model

In [14]:
mprint(chain.message_history[1].prompt)

Summarize the following in less than 20 words: "The meaning of life is a subjective question and there are various popular answers to it, such as realizing one's potential and ideals, chasing dreams, living one's dreams, or spending life for something. However, Viktor Frankl suggests that instead of asking what the meaning of life is, one should recognize that life is asking each individual and they can only answer by being responsible for their own life. Other philosophers like Arthur Schopenhauer and Friedrich Nietzsche have also provided their own perspectives on the meaning of life."

#### Second response from the chat model

In [15]:
mprint("> " + chain.message_history[1].response)

> The meaning of life is subjective, with popular answers including realizing potential and ideals, chasing dreams, and being responsible.

---